In [ ]:
import logging
from datetime import datetime, timedelta, timezone
from nost_tools import application
from nost_tools.application_utils import ConnectionConfig, ShutDownObserver
from dotenv import dotenv_values
import time

# Define the callback function
def message_callback(ch, method, properties, body):
    print(f"Received message: {body}")

def callback(ch, method, properties, body):
    print(f" [x] {method.routing_key}:{body.decode('utf-8')}")

# Set up logging
logging.basicConfig(level=logging.INFO)

# Set the prefix=exchange
PREFIX = 'nost'

# Load credentials from a .env file in current working directory
credentials = dotenv_values(".env")
HOST, PORT = credentials["HOST"], int(credentials["PORT"])
USERNAME, PASSWORD = credentials["USERNAME"], credentials["PASSWORD"]
CLIENT_ID = credentials["CLIENT_ID"]
CLIENT_SECRET_KEY = credentials["CLIENT_SECRET_KEY"]
VIRTUAL_HOST = credentials["VIRTUAL_HOST"]
IS_TLS = credentials["IS_TLS"].lower() == 'true'  # Convert to boolean

# Set the client credentials from the config file
config = ConnectionConfig(
    USERNAME,
    PASSWORD,
    HOST,
    PORT,
    CLIENT_ID,
    CLIENT_SECRET_KEY,
    VIRTUAL_HOST,
    IS_TLS)

# create the manager application from the template in the tools library
app = application.Application(app_name='test')

# add a shutdown observer to shut down after a single test case
app.simulator.add_observer(ShutDownObserver(app))

# start up the manager on PREFIX from config file
app.start_up(PREFIX, config, True)

# Send ready message
app.ready()

# Add the callback to the application
app.add_message_callback(app_name='test', app_topic='status.time', callback=callback)

# Additional logging
logging.info("Application started and callback added.")

while True:
    time.sleep(1)
